# importing libraries

In [63]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re

import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

print(os.listdir("../input"))

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)

# Any results you write to the current directory are saved as output.

['hindienglish-corpora', 'english-hindi-rnn-trainedmodelfiles']


### Load IIT-B parallel Corpus Dataset

In [64]:
# lines=pd.read_csv("../input/Hindi_English_Truncated_Corpus.csv",encoding='utf-8')
# lines = pd.read_csv("/kaggle/input/english-hindi-rnn-trainedmodelfiles/train.csv")
# dataset_used="iitb"

### Load TED Talks Parallel Corpus Dataset

In [65]:
# lines=pd.read_csv("../input/Hindi_English_Truncated_Corpus.csv",encoding='utf-8')
lines = pd.read_csv("/kaggle/input/hindienglish-corpora/Hindi_English_Truncated_Corpus.csv")
dataset_used="ted_talks"

In [66]:
if(dataset_used == "ted_talks"):
    print(lines['source'].value_counts())
    lines=lines[lines['source']=='ted']


tides        50000
ted          39881
indic2012    37726
Name: source, dtype: int64


In [67]:
lines.head(10)

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what needs to be done.,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह करने कि अनुमति नहीं है ."
1,ted,"I'd like to tell you about one such child,","मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहूंगी,"
3,ted,what we really mean is that they're bad at not paying attention.,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
7,ted,"And who are we to say, even, that they are wrong",और हम होते कौन हैं यह कहने भी वाले कि वे गलत हैं
13,ted,So there is some sort of justice,तो वहाँ न्याय है
23,ted,This changed slowly,धीरे धीरे ये सब बदला
26,ted,were being produced.,उत्पन्न नहीं कि जाती थी.
30,ted,"And you can see, this LED is going to glow.","और जैसा आप देख रहे है, ये एल.ई.डी. जल उठेगी।"
32,ted,"to turn on the lights or to bring him a glass of water,","लाईट जलाने के लिए या उनके लिए पानी लाने के लिए,"
35,ted,Can you imagine saying that?,क्या आप ये कल्पना कर सकते है


### filtering null values

In [68]:
pd.isnull(lines).sum()

source              0
english_sentence    0
hindi_sentence      0
dtype: int64

In [69]:
eng_text_col=""
hin_text_col=""
if dataset_used=="iitb":
    eng_text_col="eng_text"
elif dataset_used=="ted_talks":
    eng_text_col="english_sentence"
lines=lines[~pd.isnull(lines[eng_text_col])]
lines.head()

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what needs to be done.,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह करने कि अनुमति नहीं है ."
1,ted,"I'd like to tell you about one such child,","मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहूंगी,"
3,ted,what we really mean is that they're bad at not paying attention.,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
7,ted,"And who are we to say, even, that they are wrong",और हम होते कौन हैं यह कहने भी वाले कि वे गलत हैं
13,ted,So there is some sort of justice,तो वहाँ न्याय है


We drop the dulpicate tuples

In [70]:
lines.drop_duplicates(inplace=True)

In [71]:
# Rename the column eng_text to english_sentence and hin_text to hindi_sentence
if dataset_used=="iitb":
    lines.rename(columns={'eng_text': 'english_sentence', 'hindi_text': 'hindi_sentence'}, inplace=True)
lines.describe()

,source,english_sentence,hindi_sentence
count,38803,38803,38803
unique,1,38629,38743
top,ted,(Applause),धन्यवाद |
freq,38803,39,4


 ### Let us pick any 25000 rows from the dataset for TED talks and 1,00,000 rows for IIT-B dataset

In [72]:
if dataset_used=="iitb":
    lines=lines.sample(n=100000,random_state=42)
elif dataset_used=="ted_talks":
    lines=lines.sample(n=25000,random_state=42)
lines.shape

(25000, 3)

we convert all the english sentences to lowercase

In [73]:
# Lowercase all characters
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.lower())
# lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.lower())

remove all digits from sentences as they dont play any role in transaltion

In [74]:
# Convert to string and remove digits
# Remove all numbers from text
import string
remove_digits = str.maketrans('', '', string.digits)
lines['english_sentence'] = lines['english_sentence'].astype(str).apply(lambda x: x.translate(remove_digits))
lines['hindi_sentence'] = lines['hindi_sentence'].astype(str).apply(lambda x: x.translate(remove_digits))


remove quotes and punctions from sentences

In [75]:
# Remove quotes

lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub("'", '', x))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: re.sub("'", '', x))

In [76]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines['english_sentence']=lines['english_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

remove digits from devanagri scripts as hindi script has different digits and strip extra spaces

In [77]:


lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.translate(remove_digits))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.translate(remove_digits))

lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

# Remove extra spaces
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.strip())
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.strip())
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub(" +", " ", x))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: re.sub(" +", " ", x))


Add start and end tokens to target sequence to signal decoder to start and end

In [78]:
# Add start and end tokens to target sequences
lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x : 'START_ '+ x + ' _END')

In [79]:
lines.head()

,source,english_sentence,hindi_sentence
82040,ted,we still dont know who her parents are who she is,START_ हम अभी तक नहीं जानते हैं कि उसके मातापिता कौन हैं वह कौन है _END
85038,ted,no keyboard,START_ कोई कुंजीपटल नहीं _END
58018,ted,but as far as being a performer,START_ लेकिन एक कलाकार होने के साथ _END
74470,ted,and this particular balloon,START_ और यह खास गुब्बारा _END
122330,ted,and its not as hard as you think integrate climate solutions into all of your innovations,START_ और जितना आपको लगता है यह उतना कठिन नहीं हैअपने सभी नवाचारों में जलवायु समाधान को एकीकृत करें _END


Genrate Hind and english vocabulary by storing all the unique words in list

In [80]:
### Get English and Hindi Vocabulary
all_eng_words=set()
for eng in lines['english_sentence']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_hindi_words=set()
for hin in lines['hindi_sentence']:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

In [81]:
len(all_eng_words)

14030

In [82]:
len(all_hindi_words)

17544

calculate the length of sentences

In [83]:
lines['length_eng_sentence']=lines['english_sentence'].apply(lambda x:len(x.split(" ")))
lines['length_hin_sentence']=lines['hindi_sentence'].apply(lambda x:len(x.split(" ")))

In [84]:
lines.head()

,source,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
82040,ted,we still dont know who her parents are who she is,START_ हम अभी तक नहीं जानते हैं कि उसके मातापिता कौन हैं वह कौन है _END,11,16
85038,ted,no keyboard,START_ कोई कुंजीपटल नहीं _END,2,5
58018,ted,but as far as being a performer,START_ लेकिन एक कलाकार होने के साथ _END,7,8
74470,ted,and this particular balloon,START_ और यह खास गुब्बारा _END,4,6
122330,ted,and its not as hard as you think integrate climate solutions into all of your innovations,START_ और जितना आपको लगता है यह उतना कठिन नहीं हैअपने सभी नवाचारों में जलवायु समाधान को एकीकृत करें _END,16,20


In [85]:
lines[lines['length_eng_sentence']>30].shape

(0, 5)

filtering sentences which have less than 20 words in sentence in both the source and target language

In [86]:
lines=lines[lines['length_eng_sentence']<=20]
lines=lines[lines['length_hin_sentence']<=20]

In [87]:
lines.shape

(24774, 5)

In [88]:
print("maximum length of Hindi Sentence ",max(lines['length_hin_sentence']))
print("maximum length of English Sentence ",max(lines['length_eng_sentence']))

maximum length of Hindi Sentence  20
maximum length of English Sentence  20


In [89]:
max_length_src=max(lines['length_hin_sentence'])
max_length_tar=max(lines['length_eng_sentence'])

- input_words & target_words: Sorted lists of unique source (English) and target (Hindi, including START_/END_) vocabulary items.
- num_encoder_tokens & num_decoder_tokens: Counts of unique tokens for the encoder and decoder vocabularies, respectively.

In [90]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)
num_encoder_tokens, num_decoder_tokens

(14030, 17544)

we add 1 to decoder token for padding token

In [91]:
num_decoder_tokens += 1 #for zero padding


- input/target_token_index: Dictionaries mapping source (English) and target (Hindi) words/tokens to unique integer indices (starting from 1).
- reverse_input/target_char_index: Dictionaries mapping the integer indices back to their corresponding source and target words/tokens.

In [92]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [93]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [94]:
from sklearn.utils import shuffle
lines = shuffle(lines)
lines.head(10)

,source,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
118274,ted,helping these people to speak up their minds,START_ इसकी मदद से लोगों ने अपनी मन की बात कही _END,8,12
117489,ted,in firemen in climbers in policemen,START_ फायरब्रिगेड वालों में पर्वतारोहियों में पुलिसकर्मियों में _END,6,9
110462,ted,for us were very facile around electricity,START_ हमारे लिए बिजली से जुड़ी चीजे बहुत आसान हैं _END,7,11
109360,ted,is easy to overcome,START_ वास्तव में बहुत ही आसानी से किया जा सकता है _END,4,12
54759,ted,they want the washing machine in exactly the same way,START_ बिल्कुल उसी तरह इन्हें भी वाशिंग मशीन चाहिए _END,10,10
493,ted,this is called a cycle valve tube,START_ इसे साइकिल का वाल्व ट्यूब कहते हैं। _END,7,9
126499,ted,i wrote exactly the kinds of stories i was reading,START_ मैं वैसी ही कहानियां लिख रही थी जैसी मैं उस समय पढ़ रही थी _END,10,16
62451,ted,and what did we talk about,START_ तो हम क्या बोलते _END,6,6
86494,ted,and those top percent have been the best in the world,START_ और वो प्रतिशत दुनिया में सर्वश्रेष्ठ हैं _END,11,9
82742,ted,were going to have about eight billion perhaps more people,START_ संसार में अरब या उस से भी ज्यादा लोग होंगे _END,10,12


### Split the data into train and test

In [95]:
from sklearn.model_selection import train_test_split
X, y = lines['english_sentence'], lines['hindi_sentence']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((19819,), (4955,))

### Let us save this data

In [96]:
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')
y_test.to_pickle('y_test.pkl')

we create this function generate batches of data

In [97]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

### Encoder-Decoder Architecture

*   **Encoder:**
    *   Takes variable-length sequences of integer token indices.
    *   Embeds these indices into dense vectors (`Embedding` layer).
    *   Processes the embedded sequence with an LSTM (`encoder_lstm`).
    *   Captures the input sequence's context in its final hidden state (`state_h`) and cell state (`state_c`). These `encoder_states` are the encoder's output.

*   **Decoder:**
    *   Takes variable-length sequences of target token indices (`decoder_inputs`, typically shifted right during training).
    *   Embeds these target indices using a separate `Embedding` layer (`dec_emb_layer`).
    *   Processes the embedded sequence with its own LSTM (`decoder_lstm`).
    *   **Key Connection:** The `decoder_lstm` is initialized with the `encoder_states` (`initial_state=encoder_states`), providing it context from the source sequence.
    *   Configured with `return_sequences=True` to output a hidden state for *each* timestep.

*   **Output Layer:**
    *   A `Dense` layer with `softmax` activation is applied to the decoder LSTM's output sequence.
    *   This layer predicts the probability distribution over the entire target vocabulary (`num_decoder_tokens`) for each position in the output sequence.

*   **Training Model (`model`):**
    *   Defined with `encoder_inputs` and `decoder_inputs` as inputs.
    *   Produces `decoder_outputs` (the sequence of probability distributions).
    *   Compiled using `categorical_crossentropy` loss, suitable for comparing the predicted distributions against one-hot encoded target sequences.


In [98]:
latent_dim=300

In [99]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [100]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [101]:
from keras.optimizers import Adam
model.compile(optimizer=Adam(), loss='categorical_crossentropy')

In [102]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 300)    4209000     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 300)    5263500     input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LS

training parameters

In [103]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 100

### Early stopping and Adam Optimizer
Early Stopping:

- Prevents Overfitting: Stops training when validation performance stops improving.
- Saves Time/Resources: Avoids unnecessary epochs, potentially finding a good model faster.


Adam Optimizer:

- Adaptive Learning Rates: Calculates individual learning rates for each parameter.
- Combines Momentum & RMSprop: Leverages the strengths of both methods for often faster convergence.

In [104]:
from keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size,
                    callbacks=[early_stop])



Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/100
154/154 [==============================] - 71s 460ms/step - loss: 6.9160 - val_loss: 6.4758
Epoch 2/100
154/154 [==============================] - 68s 439ms/step - loss: 6.3541 - val_loss: 6.3458
Epoch 3/100
154/154 [==============================] - 68s 439ms/step - loss: 6.0463 - val_loss: 6.0191
Epoch 4/100
154/154 [==============================] - 67s 438ms/step - loss: 5.6991 - val_loss: 5.8179
Epoch 5/100
154/154 [==============================] - 67s 436ms/step - loss: 5.4141 - val_loss: 5.6876
Epoch 6/100
154/154 [==============================] - 68s 441ms/step - loss: 5.1651 - val_loss: 5.5695
Epoch 7/100
154/154 [==============================] - 68s 439ms/step - loss: 4.9194 - val_loss: 5.4784
Epoch 8/100
154/154 [==============================] - 67s 434ms/step - loss: 4.6884 - val_loss: 5.4086
Epoch 9/100
154/154 [=====================

- Isolates the Encoder: It creates a separate encoder_model that just takes your input sentence and produces the final "context" states (the summary of the input).
- Prepares for State Input: It sets up special input placeholders (decoder_state_input_h/c) specifically designed to receive the decoder's internal memory (states) from the previous step during generation.
- Defines One Decoder Step: It wires up the trained decoder layers (embedding, LSTM, dense) so they can take a single input word and the previous step's states, then output the probabilities for the next word and the updated states.
- Builds the Inference Decoder: It combines the single-word input, the state inputs, the single-step processing, and the outputs (probabilities + new states) into the final decoder_model used repeatedly in a loop to generate the translation word by word

In [106]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)


function take english sentence as inputs and outputs the decoded sentence in hindi

In [107]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [108]:
lines_test=X_test
lines_test.head()

10538     childrens books                                      
26526     one for yourself and one for others                  
97791     thats why in negotiations often when things get tough
102198    this is a degeneration of the retina                 
60523     i know what i have just said is simply not obvious   
Name: english_sentence, dtype: object

#### calculating bleu-1 and bleu-4 score in test split

In [109]:
# from tensorflow.keras.preprocessing.sequence import pad_sequences
X=X_test
y=y_test
# X, y = lines_test['english_sentence'], lines_test['hindi_sentence']
test_gen = generate_batch(X, y, batch_size = 1)
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
translations = []
references = []
english = []
# for idx, row in lines_test.iterrows():
#     trans = decode_sequence(row['english_sentence'])[:-4]
#     references.append(row['hindi_sentence'])
#     translated.append(trans)
for i in range(500):
    (input_seq, actual_output), _ = next(test_gen)
    trans = decode_sequence(input_seq)
    # print(X[i:i+1].values[0])
    # print(y[i:i+1].values[0][6:-4])
    # print(trans[:-4],end = "\n\n")
    references.append(y[i:i+1].values[0][6:-4])
    translations.append(trans[:-4])
    english.append(X[i:i+1].values[0])
bleu_score = corpus_bleu(
    [[ref.split()] for ref in references],  
    [trans.split() for trans in translations],
    smoothing_function=SmoothingFunction().method1
)
print(f"\nFinal BLEU-4 Score: {bleu_score:.4f}")

bleu1_score = corpus_bleu(
    [[ref.split()] for ref in references],
    [trans.split() for trans in translations],
    weights=(1, 0, 0, 0),  # Only unigram
    smoothing_function=SmoothingFunction().method1
)
print(f"BLEU-1 Score: {bleu1_score:.4f}")



Final BLEU-4 Score: 0.0200
BLEU-1 Score: 0.1899


printing some sample sentences translation

In [110]:
final_trans_df_test = pd.DataFrame({
    "en" : english,
    "hi" : references,
    "translated" : translations
})
final_trans_df_test.head()

,en,hi,translated
0,childrens books,बच्चों की किताबे,अपने पति से पूछा
1,one for yourself and one for others,एक अपने लिये और एक बाकी सब के लिये ।,एक दूसरे के लिए और अधिक कम करने के लिए
2,thats why in negotiations often when things get tough,इसिलिये जब मसले हल करते समय स्थिति कठिन हो जाती है,तो लोग लोग उन लोगों को भी नहीं कर सकते हैं
3,this is a degeneration of the retina,यह रेटिना के अध पतन है,यह एक बहुत ही एक चक्र है
4,i know what i have just said is simply not obvious,मै जानती हूँ मैने अभी जो कहा वो प्रत्यक्ष रूप से स्पष्ट नही है,मैं नहीं जानता कि मैं यह नहीं जानता कि


# Saving the model weights

In [111]:
# Saving the entire model (including architecture and weights) after training
model.save('nmt_model.h5')

# Save the encoder model (only the encoder part)
encoder_model.save('encoder_model.h5')

# Save the decoder model (only the decoder part)
decoder_model.save('decoder_model.h5')

# Optionally, save the tokenizer mappings
import pickle

# Save tokenizers (index mappings)
with open('input_token_index.pkl', 'wb') as f:
    pickle.dump(input_token_index, f)

with open('target_token_index.pkl', 'wb') as f:
    pickle.dump(target_token_index, f)

with open('reverse_input_char_index.pkl', 'wb') as f:
    pickle.dump(reverse_input_char_index, f)

with open('reverse_target_char_index.pkl', 'wb') as f:
    pickle.dump(reverse_target_char_index, f)

/opt/conda/lib/python3.6/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1_1/while/Exit_2:0' shape=(?, 300) dtype=float32>, <tf.Tensor 'lstm_1_1/while/Exit_3:0' shape=(?, 300) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '
/opt/conda/lib/python3.6/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'input_3_1:0' shape=(?, 300) dtype=float32>, <tf.Tensor 'input_4_1:0' shape=(?, 300) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


### Inference code for model TED Talks . Loads saved weights and performs inference on it. Computes Bleu scores and supports custom sentences.

In [3]:
import pickle
import numpy as np
import pandas as pd
import string
import re
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction

# Load token index files
# For IIT B dataset:
with open('/kaggle/input/english-hindi-rnn-trainedmodelfiles/input_token_index IITB.pkl', 'rb') as f:
    input_token_index = pickle.load(f)
with open('/kaggle/input/english-hindi-rnn-trainedmodelfiles/target_token_index IITB.pkl', 'rb') as f:
    target_token_index = pickle.load(f)
with open('/kaggle/input/english-hindi-rnn-trainedmodelfiles/reverse_input_char_index IITB.pkl', 'rb') as f:
    reverse_input_char_index = pickle.load(f)
with open('/kaggle/input/english-hindi-rnn-trainedmodelfiles/reverse_target_char_index IITB.pkl', 'rb') as f:
    reverse_target_char_index = pickle.load(f)

# Load encoder and decoder models
encoder_model = load_model('/kaggle/input/english-hindi-rnn-trainedmodelfiles/encoder_model IITB.h5')
decoder_model = load_model('/kaggle/input/english-hindi-rnn-trainedmodelfiles/decoder_model IITB.h5')

# Load test data
X_test = pd.read_pickle('/kaggle/input/english-hindi-rnn-trainedmodelfiles/X_test IITB.pkl')

# # For TED Talks dataset
# with open('/kaggle/input/english-hindi-rnn-trainedmodelfiles/input_token_index.pkl', 'rb') as f:
#     input_token_index = pickle.load(f)
# with open('/kaggle/input/english-hindi-rnn-trainedmodelfiles/target_token_index.pkl', 'rb') as f:
#     target_token_index = pickle.load(f)
# with open('/kaggle/input/english-hindi-rnn-trainedmodelfiles/reverse_input_char_index.pkl', 'rb') as f:
#     reverse_input_char_index = pickle.load(f)
# with open('/kaggle/input/english-hindi-rnn-trainedmodelfiles/reverse_target_char_index.pkl', 'rb') as f:
#     reverse_target_char_index = pickle.load(f)

# # Load encoder and decoder models
# encoder_model = load_model('/kaggle/input/english-hindi-rnn-trainedmodelfiles/encoder_model.h5')
# decoder_model = load_model('/kaggle/input/english-hindi-rnn-trainedmodelfiles/decoder_model.h5')

# # Load test data
# X_test = pd.read_pickle('/kaggle/input/english-hindi-rnn-trainedmodelfiles/X_test.pkl')

# Use the same max lengths as defined during training
max_length_src = 20  # From training code analysis
max_length_tar = 20  # From training code analysis

def preprocess_sentence(sentence):
    """Preprocess English sentence identically to training data"""
    sentence = sentence.lower()
    sentence = re.sub("'", '', sentence)
    sentence = ''.join(ch for ch in sentence if ch not in string.punctuation)
    remove_digits = str.maketrans('', '', string.digits)
    sentence = sentence.translate(remove_digits)
    sentence = sentence.strip()
    sentence = re.sub(" +", " ", sentence)
    return sentence

def encode_input(sentence):
    """Convert preprocessed sentence to encoder input format"""
    # Split into words and convert to token indices
    tokens = sentence.split()[:max_length_src]  # Truncate if needed
    encoded = [input_token_index.get(word, 0) for word in tokens]  # 0 for OOV/padding
    # Pad to max_length_src
    padded = pad_sequences([encoded], maxlen=max_length_src, padding='post')
    return padded

def decode_sequence(input_seq):
    """Decode sequence using trained models"""
    # Encode input to get initial states
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence with start token
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = target_token_index['START_']

    decoded_sentence = []
    stop_condition = False
    while not stop_condition:
        # Predict next token
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        
        # Sample token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = reverse_target_char_index.get(sampled_token_index, '')
        
        # Check stop condition
        if sampled_word == '_END' or len(decoded_sentence) >= max_length_tar:
            stop_condition = True
        else:
            decoded_sentence.append(sampled_word)
        
        # Update target sequence and states
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return ' '.join(decoded_sentence[:-1])  # Remove '_END' if present
translations = []
references = []




interactive translation for testing the quality of translations

In [8]:
# Assuming necessary imports and functions like preprocess_sentence, encode_input, decode_sequence are defined

def interactive_translation():
    print("Welcome to the interactive translation tool!")
    
    # Predefined sentences for testing
    predefined_sentences = [
        "what is your name ?",
        "how are you ?"
    ]
    
    # Translate predefined sentences first
    for sentence in predefined_sentences:
        print(f"Translating sentence: {sentence}")
        preprocessed = preprocess_sentence(sentence)
        encoded = encode_input(preprocessed)
        translated = decode_sequence(encoded)
        print(f"Original Sentence: {sentence}")
        print(f"Translated Sentence: {translated}")
        print("-" * 50)
    
    # Now proceed to interactive mode
    while True:
        # Step 1: Take user input (sentence to be translated)
        original_sentence = input("Enter the sentence to be translated (or type 'exit' to quit): ")
        
        # Exit condition
        if original_sentence.lower() == 'exit':
            print("Exiting the translation tool. Goodbye!")
            break
        
        # Step 2: Preprocess the sentence
        preprocessed = preprocess_sentence(original_sentence)
        
        # Step 3: Encode the preprocessed sentence
        encoded = encode_input(preprocessed)
        
        # Step 4: Translate using the model
        translated = decode_sequence(encoded)
        
        # Step 5: Print the translation
        print(f"Original Sentence: {original_sentence}")
        print(f"Translated Sentence: {translated}")
        print("-" * 50)

# Run the interactive translation tool
interactive_translation()


Welcome to the interactive translation tool!
Translating sentence: what is your name ?
Original Sentence: what is your name ?
Translated Sentence: आपका नाम क्या
--------------------------------------------------
Translating sentence: how are you ?
Original Sentence: how are you ?
Translated Sentence: तुम कैसे
--------------------------------------------------


Enter the sentence to be translated (or type 'exit' to quit):  exit


Exiting the translation tool. Goodbye!
